# pdfplumberの読み取り機能
pdfplumberにおけるPDFの読み取り範囲について知りたい  
Q1. 画像データ内のテキストは読み込めているのか？ ⇒取得できていない  
Q2. テーブル内のテキスト・数字は正しく読めているのか？⇒それなりに取得できている  
Q3. チャンクの大きさは適切か？

In [1]:
import json
import sys
import time
from operator import itemgetter
from pprint import pprint
from typing import Any, Callable, Dict, List

import pdfplumber
import requests
from dotenv import load_dotenv
from langchain import callbacks
from langchain.retrievers import EnsembleRetriever
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.retrievers import BM25Retriever
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_text_splitters import CharacterTextSplitter
from pydantic import BaseModel, Field
from sudachipy import dictionary, tokenizer

In [2]:
model = "gpt-4o-mini"
pdf_file_urls = [
    "https://storage.googleapis.com/gg-raggle-public/competitions/617b10e9-a71b-4f2a-a9ee-ffe11d8d64ae/dataset/Financial_Statements_2023.pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/617b10e9-a71b-4f2a-a9ee-ffe11d8d64ae/dataset/Hada_Labo_Gokujun_Lotion_Overview.pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/617b10e9-a71b-4f2a-a9ee-ffe11d8d64ae/dataset/Shibata_et_al_Research_Article.pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/617b10e9-a71b-4f2a-a9ee-ffe11d8d64ae/dataset/V_Rohto_Premium_Product_Information.pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/617b10e9-a71b-4f2a-a9ee-ffe11d8d64ae/dataset/Well-Being_Report_2024.pdf",
]

In [4]:
# 戻り値として質問に対する回答を返却してください。
def download_and_load_pdfs(urls: list) -> list:
    """
    PDFファイルをダウンロードして読み込む関数

    Args:
        urls (list): PDFファイルのURLリスト

    Returns:
        documents (list): PDFファイルのテキストデータを含むDocumentオブジェクトのリスト

    Raises:
        Exception: ダウンロードまたは読み込みに失敗した場合に発生する例外

    Examples:
        >>> urls = ["https://example.com/example.pdf"]
        >>> download_and_load_pdfs(urls)
        [Document(page_content="...", metadata={"source": "https://example.com/example.pdf"})]
    """
    try:

        def download_pdf(url, save_path):
            response = requests.get(url)
            if response.status_code == 200:
                with open(save_path, "wb") as f:
                    f.write(response.content)
            else:
                raise Exception(f"Failed to download {url}")

        documents = []

        for i, url in enumerate(urls):
            tmp_path = f"pdf_{i}.pdf"
            download_pdf(url, tmp_path)

            with pdfplumber.open(tmp_path) as pdf:
                full_text = ""
                for page in pdf.pages:
                    text = page.extract_text()
                    if text:
                        full_text += text + "\n"

                documents.append(Document(page_content=full_text, metadata={"source": url}))
        return documents
    except Exception as e:
        raise Exception(f"Error reading {url}: {e}")


docs = download_and_load_pdfs(pdf_file_urls)

In [ ]:
print("Financial_Statements_2023.pdf")
pprint(docs[0].page_content)

In [18]:
print("Hada_Labo_Gokujun_Lotion_Overview.pdf")
pprint(docs[1].page_content)

Hada_Labo_Gokujun_Lotion_Overview.pdf
('2024/12/06 8:55 肌ラボ 極潤ヒアルロン液 | ロート製薬: 商品情報サイト\n'
 'アイケア スキンケア 外⽪薬 内服薬 検査薬 ⿐・⼝腔ケア ⾷品・サプリメント お役⽴ち情報\n'
 '肌ラボ\n'
 '化粧品\n'
 '肌ラボ 極潤ヒアルロン液\n'
 '発酵技術で⽣まれたうるおい成分 乳酸球菌／ヒア\n'
 'ルロン酸発酵液 配合。やさしいとろみでうるおっ\n'
 'て吸いつくようなもちもち肌に\n'
 '内容量 価格\n'
 'ボトル 170mL オープン価格\n'
 'つめかえ⽤ 170mL オープン価格\n'
 'ポンプ 400mL オープン価格\n'
 '※希望⼩売価格・税込\n'
 '肌ラボ 極潤ヒアルロン液の成分表⽰はこちらをご覧ください。\n'
 '成分表⽰ 販売店検索はこちら\n'
 'JANコード\n'
 '肌ラボ 極潤ヒアルロン液について\n'
 '肌が本当に求めるうるおいを追求した、「極潤」。うるおい成分であるヒアルロン酸にこだわり抜き、必要のない\n'
 'ものはできる限り削ぎ落として、配合成分と容器をシンプルにしました。\n'
 '「極潤ヒアルロン液」は、肌が求めるうるおいを詰めこんだ化粧⽔。4種類のヒアルロン酸（うるおい成分）＊\n'
 'を配合しました。やさしいとろみで、たっぷりうるおい、使うたび、健やかな素肌へ。顔・体・髪の⽑にも。⽼\n'
 '若男⼥、誰でも使えます。\n'
 '弱酸性。無⾹料・無着⾊・オイルフリー・アルコール（エタノール）フリー・パラベンフリー。敏感肌の⽅の協\n'
 '⼒によるパッチテスト済み。（すべての⽅に刺激が起こらないというわけではありません。）\n'
 '本製品の容器には、環境に配慮したバイオマス原料を⼀部使⽤しています。\n'
 '＊：加⽔分解ヒアルロン酸（ナノ化ヒアルロン酸）、アセチルヒアルロン酸Na（スーパーヒアルロン酸）、乳酸球菌／ヒアルロン酸発\n'
 '酵液（乳酸発酵ヒアルロン酸）、ヒアルロン酸Na\n'
 '◆本品は、航空法で定める航空危険物に該当しません。\n'
 '★販売名：ハダラボモイスト化粧⽔d\n'
 '使⽤上の注意\n'
 '＜相談すること＞\n'
 '○肌に異常が⽣じていない

In [20]:
print("Shibata_et_al_Research_Article.pdf")
pprint(docs[2].page_content)

Shibata_et_al_Research_Article.pdf
('Resource\n'
 'Selective Laminin-Directed Differentiation of Human\n'
 'Induced Pluripotent Stem Cells into Distinct Ocular\n'
 'Lineages\n'
 'Graphical Abstract Authors\n'
 'ShunShibata,RyuheiHayashi,\n'
 'ToruOkubo,...,AndrewJ.Quantock,\n'
 'KiyotoshiSekiguchi,KohjiNishida\n'
 'Correspondence\n'
 'ryuhei.hayashi@ophthal.med.osaka-u.\n'
 'ac.jp(R.H.),\n'
 'knishida@ophthal.med.osaka-u.ac.jp\n'
 '(K.N.)\n'
 'In Brief\n'
 'Shibataetal.reportthatlamininisoforms\n'
 'differentiallyregulatetheocularcell\n'
 'differentiationfromhiPSCs.Thebinding\n'
 'affinityoflamininandintegrins\n'
 'determinesthenatureofexpandedhiPSC\n'
 'coloniesintermsofcellmotility,cell-cell\n'
 'interactions,andcelldensity,withthe\n'
 'involvementofWntandYAPsignals.\n'
 'Highlights\n'
 'd LN211E8promoteshiPSCdifferentiationintoneuralcrest\n'
 'cellsviaWntactivation\n'
 'd LN332E8promoteshiPSCdifferentiationintocorneal\n'
 'epithelialcells\n'
 'd LN511E8ledtoYAPinactivationandretinal

In [19]:
print("V_Rohto_Premium_Product_Information.pdf")
pprint(docs[3].page_content)

V_Rohto_Premium_Product_Information.pdf
''


In [21]:
print("Well-Being_Report_2024.pdf")
pprint(docs[4].page_content)

Well-Being_Report_2024.pdf
('ROHTO Well-being Report\n'
 '2024\n'
 'ロート製薬 統合レポート\n'
 '経営理念\n'
 '「健康」という価値提供を通じて\n'
 '豊かで幸せな生活を送るための心身の健康に貢献し続けることが\n'
 'ロートの最大の責務と捉え、その実現のために\n'
 '長期視点での経営と価値創出に努める\n'
 '世界の人々を\n'
 'ロートは、社会の公器としての使命を自覚し、\n'
 'ロートを取りまくすべての人たちと協働して社会課題を解決し、\n'
 'これにより得られた便益を共有する\n'
 'Well-being\n'
 '「\n'
 '」\n'
 '定款 第一章 第二条\n'
 'に\n'
 'ロートグループ総合経営ビジョン2030\n'
 '-Connect for Well-being-\n'
 'ロート製薬は、創業以来「健康」をコアバリューに、\n'
 'Well-beingとは身体も心もイキイキとし、 一般用医薬品やスキンケア商品の提供を通じて、\n'
 'さまざまなライフステージにおいて笑顔あふれる幸せな毎日を過ごすこと 多くの方に身近な「健康」をお届けしてまいりました。\n'
 '生活者の皆さま一人ひとりの健康寿命が延伸し、\n'
 'ロート製薬は、世界の人々がWell-beingを実感できる時間が少しでも長くなるように、\n'
 '生活の質（Quality of Life）が向上することによって、\n'
 '医薬品、スキンケアに加え、さまざまな事業で\n'
 '社会全体の経済活動は活性化し、増加する社会保障費も抑制され、\n'
 'イノベーションを起こし幅広く「つなげていく」\n'
 '持続的な健康長寿社会の実現につながると考えています。\n'
 'それを実現するために、社内外の仲間同士を、組織と組織を\n'
 '当社の存在意義（パーパス）は、\n'
 'しっかり「つなげていく」\n'
 'そして、信頼の連鎖の上に人材を育成し、一体感のある組織を作り上げ 世界の人々に商品やサービスを通じて「健康」をお届けすることによって、\n'
 '人々の更なるWell-beingに「つなげていく」 当社を取り巻くすべての人や社会を「Well-being」へと導き、\n

# PyMuPDFの読み取りテスト

# easyOCRを使って画像からテキストを抽出

In [4]:
import fitz  # PyMuPDF
import easyocr
import os


def download_pdf(url, save_path):
    response = requests.get(url)
    if response.status_code == 200:
        with open(save_path, "wb") as f:
            f.write(response.content)
    else:
        raise Exception(f"Failed to download {url}")


def pdf_to_images(pdf_path, output_folder="pdf_images"):
    """
    PDFの各ページを画像として保存する関数。

    Args:
        pdf_path (str): PDFファイルのパス。
        output_folder (str): 画像を保存するフォルダ名。

    Returns:
        list: 保存された画像ファイルのパスリスト。
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    pdf_document = fitz.open(pdf_path)
    image_paths = []

    for page_num in range(len(pdf_document)):
        page = pdf_document[page_num]
        pix = page.get_pixmap()
        image_path = os.path.join(output_folder, f"page_{page_num + 1}.png")
        pix.save(image_path)
        image_paths.append(image_path)

    pdf_document.close()
    return image_paths


def extract_text_from_images(image_paths):
    """
    画像からテキストを抽出する関数。

    Args:
        image_paths (list): 画像ファイルのパスリスト。

    Returns:
        str: 抽出されたテキストを結合した文字列。
    """
    reader = easyocr.Reader(["ja", "en"])  # 日本語と英語をサポート
    extracted_text = ""

    for image_path in image_paths:
        print(f"Processing: {image_path}")
        results = reader.readtext(image_path)
        for _, text, _ in results:
            extracted_text += text + "\n"

    return extracted_text


def pdf_text_extraction_pipeline(pdf_path):
    """
    PDFファイルからテキストを抽出するパイプライン。

    Args:
        pdf_path (str): PDFファイルのパス。

    Returns:
        str: PDF全体から抽出されたテキスト。
    """
    print("Converting PDF to images...")
    image_paths = pdf_to_images(pdf_path)

    print("Extracting text from images...")
    extracted_text = extract_text_from_images(image_paths)

    return extracted_text


# 実行例
pdf_path = "/root/git_work/90_adhoc/RagRohtoCompetition/notebook/pdf_3.pdf"
extracted_text = pdf_text_extraction_pipeline(pdf_path)

# 抽出されたテキストを表示
print("Extracted Text:")
print(extracted_text)

Converting PDF to images...
Extracting text from images...
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% CompleteProcessing: pdf_images/page_1.png
Processing: pdf_images/page_2.png
Extracted Text:
眞用前にこのに甲車をどすおりみくたさい.支た,む‥なときにオめるよう原@してくたさい. 因・亜
浜恋名'vロートブレミアム』
宮2  三名
ゆ-
ロート史上最高峰処方。
PRENlUM 醐使して番積したつらい療れ目を治す!
国内最多12有効成分配合
歌人の硬れ巨と し,その面因に多アカにアブローア 。
角膜保謹威分高分子コンドロイチン配合
ルーフーーーー・イリトフテルー-リウ
ウ郎す|って保}し 良れ*}@します』
一y -川ゥとい*・こセこル ・・ そerozronいヶw-~ッい川 ‥ アルルー』
とりやばりここちこしたりい
3っル
多ぶ三式
ハつ川
十か一
ひかェにカー ニロし・ロにていき
ロッーアーッアて・て こ-ていり川こ】
〈川・ロ
さ用上山
~ら
がゆサかちらたれんセこけワ・爪町
相盛互ること
町やか上石ッこ
口っにに川ケカリし このに
ゆu[ク川-に下
二ルつル
リールー-一三川川
丘 仮えせにこ相 < たさい:
打たさい
【にけひたいだににていュノ
山W
一門川
無城をにこし
兄り 兄加 かゆみ
ルーーーリーケ
アのたがの田なり  けいい日の町田
_.ざや丞にな_し己と‥
【川~町ににけに .  円〉
ぶの川戸は屋川を中止し
この引をけっこ
〈口は ーこ川さり
馬器ポ愛荒,議良ならない場合
屈志 屈立
1回1二-_15
え回点眼してください』
川ラキ
お_に屋いする_
ぶちしほuしこたりじにりざむて元口をゼ: こと
「「り;すいじはき 市た『 し‥ にさい
りに・にやす
うにに …にラにに か町いヶにやきたて
りにさい
こ先を口にきのに
よとにきりさたないで: にこい はちに戸バ向
~一
jntie_『おににもqしにき"ふり しない
つ‥にきい
・のうと
は ‥ リコーコーー[ニ「-)I…「けレり
日コ十り
り!に川てい さい
ルワ・ニヒローいコに
に百
はたらき
コテトラヒトロ」リン
りワの
の口を山ゆさせこに元町を子すつ
ネナス

In [2]:
import fitz  # PyMuPDF
import easyocr
import os


def pdf_to_images(pdf_path, output_folder="pdf_images", dpi=300):
    """
    PDFの各ページを高解像度の画像として保存する関数。

    Args:
        pdf_path (str): PDFファイルのパス。
        output_folder (str): 画像を保存するフォルダ名。
        dpi (int): 画像解像度（ドット・パー・インチ）。

    Returns:
        list: 保存された画像ファイルのパスリスト。
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    pdf_document = fitz.open(pdf_path)
    image_paths = []

    for page_num in range(len(pdf_document)):
        page = pdf_document[page_num]
        # DPIに基づいた解像度で画像を生成
        matrix = fitz.Matrix(dpi / 72, dpi / 72)
        pix = page.get_pixmap(matrix=matrix)
        image_path = os.path.join(output_folder, f"page_{page_num + 1}.png")
        pix.save(image_path)
        image_paths.append(image_path)

    pdf_document.close()
    return image_paths


def extract_text_from_images(image_paths):
    """
    画像からテキストを抽出する関数。

    Args:
        image_paths (list): 画像ファイルのパスリスト。

    Returns:
        str: 抽出されたテキストを結合した文字列。
    """
    reader = easyocr.Reader(["ja", "en"])  # 日本語と英語をサポート
    extracted_text = ""

    for image_path in image_paths:
        print(f"Processing: {image_path}")
        results = reader.readtext(image_path)
        for _, text, _ in results:
            extracted_text += text + "\n"

    return extracted_text


def pdf_text_extraction_pipeline(pdf_path, dpi=300):
    """
    PDFファイルからテキストを抽出するパイプライン。

    Args:
        pdf_path (str): PDFファイルのパス。
        dpi (int): 画像の解像度（ドット・パー・インチ）。

    Returns:
        str: PDF全体から抽出されたテキスト。
    """
    print("Converting PDF to images...")
    image_paths = pdf_to_images(pdf_path, dpi=dpi)

    print("Extracting text from images...")
    extracted_text = extract_text_from_images(image_paths)

    return extracted_text


# 実行例
pdf_path = "/root/git_work/90_adhoc/RagRohtoCompetition/notebook/pdf_3.pdf"
extracted_text = pdf_text_extraction_pipeline(pdf_path, dpi=300)

# 抽出されたテキストを表示
print("Extracted Text:")
print(extracted_text)

Converting PDF to images...
Extracting text from images...
Processing: pdf_images/page_1.png
Processing: pdf_images/page_2.png
Extracted Text:
使用前にこの説明書を必ずお読みください。また 必要なときに読めるよう保管してください。
園n雨繁
販売名:Vロートプレミアム回
第2類医薬品
V
ロート史上最高峰処方。
PREMIUM `使して馨積したつらい渡れ目を治す!
国内最多12有効成分配合
(基準米内)
現代人の渡れ目を科学し その原因に多角的にアプローチ。
角膜保護成分高分子コンドロイチン配合
(コンドロイチン硫酸エステルナトリウム)
角膜を覆って保護し渡れを改善しますc
※ロート製薬の目薬として最多有効成分配合 *基準とは厚生労働省が承認事務の効率化を図るために定めた医薬品の範囲
どの角度からでもさしやすい
あ
フリー
ワンタッチ式
アングル
スクリュー
ノズルG
キヤップ
開ける時は左に1回カチッと回し閉める時も
簡単にアイケアできて とっても便利です
右に1回カチッと回すだけ。簡単便利です_
使用上の注意
包使用後 次の症状があらわれた場合は副作用の
相談すること
可能性があるので 直ちに使用を中止し この説
1.次の人は使用前に医師;
薬剤師又は
明書を持って医師 薬剤師又は登録販売者にご
登録販売者にご相談ください。
相談ください。
(1)医師の治療を受けている人
関係部位
症状
(P)薬などによりアレルギー症状を起こし
皮ふ
発療・発赤 かゆみ
たことがある人
目
充血 かゆみ はれ しみて痛し)
(3)次の症状のある人:はげしい目の痛み
(4)次の診断を受けた人:緑内障
3.次の場合は使用を中止し、 この説明書を持って
医師 薬剤師又は登録販売者にご相談ください。
{影蔵擬慧艶わ撮ならない場合
用法・用量
1回1~2浦
1日5~6回点眼してください。
用法・用量に関連する注意
(1)過度に使用すると 異常なまぶしさを感じたりかえって充血を摺くこと
がありますので用法 用量を厳守してくださし。
(P)小児に使用させる場合には 保護者の指導監督のもとに使用させて
くださし。
(3)容器の先を目やまぶた まつ毛に触れ

In [21]:
image = cv2.imread("/root/git_work/90_adhoc/RagRohtoCompetition/notebook/pdf_images/page_1.png")
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # 画像をグレースケールに変換
_, image = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY)  # グレースケール画像を2値化

reader = easyocr.Reader(["ja", "en"])
easyocr_text = reader.readtext(image, detail=0)
print(f"easyocr_text: {easyocr_text}")

easyocr_text: ['~上"; : …、', "5充乃'BE", 'ブレミアム・ー', '"こぶ', 'W-ト', 'ロート史上展高峰処方。', "'‥ト川いKi", '蒔康して替精したつらい療れ目を治す', '国内最多12有効成分亜合', '双ヶ人の延てEとバ学: こゆ万因に多爪河アふい-ご', '角膜保盛威分向分子コンドロイチン配台', '‥川エセ[ こニル ナ', '出 * てに出に 白れ*中三し天丁;', 'アンゴル', '翠等*', 'ば二川', '他用上のi菌', '+継 \'に1m"+らわさ上州とは川', '花 当石こと', 'よえぃじ', 'り『爪ヶカト', 'ツ', '次『」ル`目沙 -', 'ぶ川こは', '小ブ5病っこ氏江', 'さけ =「川や すや', '留避具崎村二', '物【"ださい', '卯に円川', '* * *', '…', '"し芹は凪川を ド止L', '二川葉日モけ_こ', 'ヶけ せ子にかりりはこりルーヱにー払に…さり', 'w志 ', '1回1 こ恵;', '1日5--回点駅してくださいッ', '団「 川川t =キや', 'i;', '‥さうここついニに', ' …']
